In [183]:
import requests
from bs4 import BeautifulSoup
import pandas
import numpy as np

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'  
}

In [101]:
stock_id = 6257

In [189]:
def get_cash_dividend_payouts( stock_id, 
                               year_to_check=5):
    res = requests.get("https://histock.tw/stock/{}/%E9%99%A4%E6%AC%8A%E9%99%A4%E6%81%AF".format(stock_id), headers = headers)
    res.encoding = 'utf-8'
    
    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pandas.read_html(data.prettify())

    df = dfs[0]
    df = df[1:]
    
    cash_dividend_payouts = df["現金股利"][:year_to_check].astype('float')
    return cash_dividend_payouts

In [239]:
def get_ROEs( stock_id, 
              year_to_check=5):
    
    res = requests.get("https://histock.tw/stock/financial.aspx?no={}&t=3&st=2&q=3".format(stock_id), headers = headers)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pandas.read_html(data.prettify())

    df = dfs[0]
    ROEs = [float(ROE.strip('%')) for ROE in df["年度ROE"][:year_to_check]]
    return ROEs

In [206]:
def get_PEratios( stock_id,
                  year_to_check=20):
    
    res = requests.get("https://histock.tw/stock/{}/%E6%9C%AC%E7%9B%8A%E6%AF%94".format(stock_id), headers = headers)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pandas.read_html(data.prettify())

    df = dfs[0]
    df = df[1:]
    
    PEratio_cols = [col for col in df.columns if '本益比' in col]
    PEratios = []
    for col in PEratio_cols:
        PEratios += list(df[col])

    return PEratios[:year_to_check]

In [211]:
def get_prev_net_val_per_share( stock_id, 
                                prev_year="2019"):

    res = requests.get("https://histock.tw/stock/{}/%E6%AF%8F%E8%82%A1%E6%B7%A8%E5%80%BC".format(stock_id), headers = headers)
    res.encoding = 'utf-8'

    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")
    dfs = pandas.read_html(data.prettify())

    df = dfs[0]

    prev_net_val_per_share = df.iloc[3][prev_year]
    return prev_net_val_per_share

In [243]:
def get_EPSs( stock_id, 
              year_to_check=5):
    
    res = requests.get("https://histock.tw/stock/{}/%E9%99%A4%E6%AC%8A%E9%99%A4%E6%81%AF".format(stock_id), headers = headers)
    res.encoding = 'utf-8'
    
    soup = BeautifulSoup(res.text, 'lxml')
    data = soup.select_one("table", class_="tb-outline")

    dfs = pandas.read_html(data.prettify())

    df = dfs[0]
    df = df[1:]

    EPSs = df["EPS"][:year_to_check].astype('float')
    return list(EPSs)

In [244]:
cash_dividend_payouts = get_cash_dividend_payouts( stock_id, year_to_check=5 )
EPSs = get_EPSs( stock_id, year_to_check=5 )
ROEs = get_ROEs( stock_id, year_to_check=5 )
PEratios = get_PEratios( stock_id, year_to_check=20 )
prev_net_val_per_share = get_prev_net_val_per_share( stock_id, prev_year="2019" )

In [245]:
print (EPSs)
print (ROEs)
print (PEratios)
print (prev_net_val_per_share)

[3.26, 3.01, 2.76, 2.17, 2.11]
[12.31, 12.13, 12.95, 10.03, 9.4]
[9.86, 9.96, 12.85, 11.94, 11.0, 11.88, 10.05, 12.68, 12.79, 13.79, 12.97, 13.91, 12.47, 9.9, 10.1, 9.74, 10.48, 10.17, 8.65, 8.14]
26.75
